# Model nondeterminism (cursory examination)

SPDX-License-Identifier: 0BSD

Embedding the same text with the same model multiple times sometimes returns
different results. This is okay, since the results are sufficiently similar.

This also shows one way to serialize and deserialize embeddings with the
safetensors library.

In [1]:
import logging

import numpy as np
import openai
import openai.embeddings_utils
import safetensors.numpy

In [2]:
logging.basicConfig(level=logging.INFO)
openai.api_key_path = '.api_key'

In [3]:
def save(embeddings, basename):
    safetensors.numpy.save_file(
        tensor_dict={'embeddings': embeddings},
        filename=f'{basename}.safetensors',
    )

In [4]:
def load(basename):
    tensor_dict = safetensors.numpy.load_file(f'{basename}.safetensors')
    return tensor_dict['embeddings']

In [5]:
def embed(texts):
    embeddings = openai.embeddings_utils.get_embeddings(
        texts,
        engine='text-embedding-ada-002',
    )
    return np.array(embeddings, np.float32)

In [6]:
TEXTS = [
    'Gee whiz!',
    'Golly wow!',
    'Well, shucks!',
    'The meeting is this afternoon!',
]

In [7]:
a = embed(TEXTS)

In [8]:
a @ np.transpose(a)

array([[0.9999997 , 0.92304474, 0.880876  , 0.7803911 ],
       [0.92304474, 0.9999997 , 0.8696495 , 0.7692273 ],
       [0.880876  , 0.8696495 , 0.9999999 , 0.7567709 ],
       [0.7803911 , 0.7692273 , 0.7567709 , 0.9999999 ]], dtype=float32)

In [9]:
b = embed(TEXTS)

In [10]:
(a == b).all()

False

In [11]:
a - b

array([[-1.6696658e-04,  1.3367026e-03,  1.2473017e-04, ...,
        -1.6386993e-04, -6.8068644e-04,  7.0284493e-04],
       [-9.1372058e-05, -5.7785423e-05, -2.5609508e-05, ...,
         2.5652349e-05, -6.3115731e-06, -5.2718446e-05],
       [ 2.0223390e-04, -2.8806739e-04,  1.6142055e-04, ...,
         1.8481724e-04,  2.3951288e-04,  2.4639070e-05],
       [ 1.9489788e-05,  2.1886081e-08, -2.6035123e-05, ...,
         1.6950071e-07, -2.9133749e-05,  1.8812716e-07]], dtype=float32)

In [12]:
(a - b).min(), (a - b).max()

(-0.0026937034, 0.0031313733)

In [13]:
abs(a - b).mean()

0.00022494205

In [14]:
[np.dot(a_row, a_row) for a_row in a]

[1.0, 1.0000001, 0.99999976, 1.0]

In [15]:
[np.dot(b_row, b_row) for b_row in b]

[0.99999994, 1.0, 1.0, 1.0]

In [16]:
[np.dot(a_row, b_row) for a_row, b_row in zip(a, b)]

[0.99950594, 0.9999989, 0.99995404, 0.99999845]

In [17]:
save(a, 'a')
save(b, 'b')